In [1]:
import pandas as pd
import sys
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from IPython.display import display, HTML
from scipy.stats import spearmanr
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import zipfile
import random
import gzip
import sys
import h5py
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.width", 2000)
pd.set_option("display.max_colwidth", None)

display(HTML("""
<style>
.dataframe td, .dataframe th {
    white-space: nowrap !important;
}
</style>
"""))

In [2]:
# Define root directory
# root = os.path.dirname(os.path.abspath(__file__))
root = "."
sys.path.append(os.path.join(root, "..", "src"))
from default import DATAPATH, CONFIGPATH

# Load pathogen info
# pathogen_code = sys.argv[1]
pathogen_code = 'mtuberculosis'
df = pd.read_csv(os.path.join(CONFIGPATH, 'pathogens.csv'))
row = df.loc[df["code"].eq(pathogen_code)]
if row.empty: 
    raise SystemExit(f"Unknown code: {pathogen_code}")
pathogen = row.iloc[0]["pathogen"]

print("Step XX")

# Define output directory
OUTPUT = os.path.join(root, "..", "output")

# Shared columns
KEYS = ["assay_id", "activity_type", "unit"]

# Columns to take from datasets and clusters tables
COLUMNS_DATASETS = ["equal", 'higher', 'lower', "target_type_curated_extra", "dataset_type", "cpds_qt", "min_", "p1", "p25", "p50", "p75", "p99", "max_", "pos_ql", "ratio_ql", "cpds_ql"]
COLUMNS_CLUSTERS = ['clusters_0.3', 'clusters_0.6', 'clusters_0.85']

Step XX


In [3]:
def load_all_gz_csvs_from_zip(zip_path):
    """Read all ``*.csv.gz`` members from a ZIP into DataFrames.

    Parameters
    ----------
    zip_path : str | pathlib.Path
        Path to the ZIP archive.

    Returns
    -------
    dict[str, pandas.DataFrame]
        Mapping of ZIP member name -> loaded DataFrame.
    """
    dfs = {}
    with zipfile.ZipFile(zip_path, "r") as z:
        for name in z.namelist():
            if name.endswith(".csv.gz"):
                with z.open(name) as f:
                    dfs[name] = pd.read_csv(f, compression="gzip")
    return dfs

def get_all_results_from_individual_modeling(INDIVIDUAL_LM, LABELS=['A', 'B', 'C', 'D']):
    """Collect best AUROC (>0.7) per (assay_id, activity_type, unit) for each label.

    Returns
    -------
    RESULTS : dict[str, dict[tuple, list]]
        Per label: (assay_id, activity_type, unit) -> [expert_cutoff, best_auroc],
        considering only rows with AUROC > 0.7.
    CONSIDERED_ASSAYS : dict[str, set[tuple]]
        Per label: set of all (assay_id, activity_type, unit) keys encountered
        (no AUROC threshold applied).
    """
    RESULTS, CONSIDERED_ASSAYS = {}, {}
    for LABEL in LABELS:
        RESULTS[LABEL] = {}
        CONSIDERED_ASSAYS[LABEL] = set()
        rows = INDIVIDUAL_LM[INDIVIDUAL_LM[LABEL]][["assay_id", "activity_type", "unit", "expert_cutoff", f"{LABEL}_AVG"]].values
        for assay_id, activity_type, unit, expert_cutoff, auroc in rows:
            key = (assay_id, activity_type, unit)
            CONSIDERED_ASSAYS[LABEL].add(key)
            if auroc > 0.7:
                if key not in RESULTS[LABEL]:
                    RESULTS[LABEL][key] = [expert_cutoff, auroc]
                elif auroc > RESULTS[LABEL][key][1]:
                    RESULTS[LABEL][key] = [expert_cutoff, auroc]
    return RESULTS, CONSIDERED_ASSAYS

def where_considered(key, LABELS, CONSIDERED_ASSAYS):
    """Return labels (semicolon-separated) where `key` was considered; else NaN."""
    considered = []
    for LABEL in LABELS:
        if key in CONSIDERED_ASSAYS[LABEL]:
            considered.append(LABEL)
    if len(considered) > 0:
        return ";".join(considered)
    else:
        return np.nan
    
def where_accepted(key, LABELS, ACCEPTED_ASSAYS):
    """Return labels (semicolon-separated) where `key` was accepted; else NaN."""
    accepted = []
    for LABEL in LABELS:
        if key in ACCEPTED_ASSAYS[LABEL]:
            accepted.append(LABEL)
    if len(accepted) > 0:
        return ";".join(accepted)
    else:
        return np.nan

def get_filtered_assay_master(assay_df, activity_type, unit, target_type_curated_extra, bao_label, strain):
    """Filter `assay_df` by metadata fields, treating non-string `unit` as missing (NaN)."""
    if type(unit) == str:
        df = assay_df[(assay_df['activity_type'] == activity_type) & 
                    (assay_df['unit'] == unit) &
                    (assay_df['target_type_curated_extra'] == target_type_curated_extra) &
                    (assay_df['bao_label'] == bao_label) &
                    (assay_df['strain'] == strain)]
    else:
        df = assay_df[(assay_df['activity_type'] == activity_type) & 
                    (assay_df['unit'].isna()) &
                    (assay_df['target_type_curated_extra'] == target_type_curated_extra) &
                    (assay_df['bao_label'] == bao_label) &
                    (assay_df['strain'] == strain)]
    return df

def load_expert_cutoffs(CONFIGPATH):
    """
    Load expert cutoffs from the manual curation CSV and return them as a dictionary.

    The CSV is expected at:
        {CONFIGPATH}/manual_curation/expert_cutoffs.csv

    The returned dictionary maps:
        (activity_type, unit, target_type, pathogen_code) -> expert_cutoff

    Parameters
    ----------
    CONFIGPATH : str
        Path to the config folder.

    Returns
    -------
    dict
        Dictionary of expert cutoffs keyed by
        (activity_type, unit, target_type, pathogen_code).
    """
    # Load expert cut-offs
    EXPERT_CUTOFFS = pd.read_csv(os.path.join(CONFIGPATH, "expert_cutoffs.csv"))

    EXPERT_CUTOFFS = {
        (a, b, c, d): [float(k) for k in e.split(";")]
        for a, b, c, d, e in EXPERT_CUTOFFS[
            ["activity_type", "unit", "target_type", "pathogen_code", "expert_cutoff"]
        ].values
    }

    return EXPERT_CUTOFFS

def load_ecfp_all(h5_path):
    """Load all ECFP (Morgan count) fingerprints.

    Parameters
    ----------
    h5_path : str
        Path to the HDF5 file containing datasets "SMILES" and "X_morgan".

    Returns
    -------
    dict[str, np.ndarray]
        Mapping {chembl_id: fingerprint (np.int8, shape (nBits,))}.
    """
    with h5py.File(h5_path, "r") as f:
        meta = f["SMILES"][:, 3].astype(str)
        fps  = f["X_morgan"][:]  # Load ALL

    return {cid: fp for cid, fp in zip(meta, fps)}

def KFoldTrain(X, Y, n_splits=4, n_estimators=100, random_state=42):
    """Stratified K-fold training/eval with RandomForest; returns mean AUROC and std.

    Parameters
    ----------
    X : np.ndarray
        Feature matrix (n_samples, n_features).
    Y : np.ndarray
        Binary labels (n_samples,).
    n_splits : int
        Number of folds.
    n_estimators : int
        Number of trees in the random forest.
    random_state : int
        RNG seed (also used for fold shuffling).

    Returns
    -------
    tuple[float, float]
        (mean_auroc, std_auroc) rounded to 3 decimals.
    """
    def init_RF():
        return RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features="sqrt",
            n_jobs=8,
            random_state=random_state,
        )

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    aurocs = []

    for train_idx, test_idx in skf.split(X, Y):
        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        rf = init_RF()
        rf.fit(X_train, Y_train)
        y_prob = rf.predict_proba(X_test)[:, 1]
        aurocs.append(roc_auc_score(Y_test, y_prob))

    return round(float(np.mean(aurocs)), 3), round(float(np.std(aurocs)), 3)

def TrainRF(X, Y, n_estimators=100):
    """Train a RandomForestClassifier on all provided data and return the fitted model.

    Parameters
    ----------
    X : np.ndarray
        Feature matrix (n_samples, n_features).
    Y : np.ndarray
        Labels (n_samples,).

    Returns
    -------
    RandomForestClassifier
        Fitted classifier.
    """
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        n_jobs=8,
    )
    rf.fit(X, Y)
    return rf


In [ ]:
# Load assays info
ASSAYS_CLEANED = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_cleaned.csv"))
ASSAYS_CLUSTERS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_clusters.csv"))
ASSAYS_PARAMETERS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_parameters.csv"))
ASSAYS_DATASETS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_datasets.csv"))
INDIVIDUAL_LM = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "individual_LM.csv"))
MERGED_LM = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "merged_LM.csv"))

# Getting keys
KEYS = ["ssay_id", "activity_type", "unit"]

# Merge everything
ASSAYS_MASTER = ASSAYS_CLEANED.merge(ASSAYS_PARAMETERS,on=KEYS, how="left", validate="1:1")


In [22]:
# Assays considered at some point
considered_A = set([tuple(i) for i in INDIVIDUAL_LM[INDIVIDUAL_LM['A']][KEYS].values])
considered_B = set([tuple(i) for i in INDIVIDUAL_LM[INDIVIDUAL_LM['B']][KEYS].values])
considered_C = set([tuple(i) for i in INDIVIDUAL_LM[INDIVIDUAL_LM['C']][KEYS].values])
considered_D = set([tuple(i) for i in INDIVIDUAL_LM[INDIVIDUAL_LM['D']][KEYS].values])

# Assays accepted at some point
accepted_A = set([tuple(i) for i in INDIVIDUAL_LM[INDIVIDUAL_LM['A_AVG'] > 0.7][KEYS].values])
accepted_B = set([tuple(i) for i in INDIVIDUAL_LM[INDIVIDUAL_LM['B_AVG'] > 0.7][KEYS].values])
accepted_C = set([tuple(i) for i in INDIVIDUAL_LM[INDIVIDUAL_LM['C_AVG'] > 0.7][KEYS].values])
accepted_D = set([tuple(i) for i in INDIVIDUAL_LM[INDIVIDUAL_LM['D_AVG'] > 0.7][KEYS].values])

In [23]:
print(len(accepted_A))
print(len(accepted_B))
print(len(accepted_C))
print(len(accepted_D))

12
0
19
19


In [21]:
INDIVIDUAL_LM

,assay_id,activity_type,unit,target_type,target_type_curated_extra,cpds,direction,dataset_type,expert_cutoff,pos_qt,ratio_qt,cpds_qt,pos_ql,ratio_ql,cpds_ql,A,A_AVG,A_STD,B,B_AVG,B_STD,C,C_AVG,C_STD,D,D_AVG,D_STD
0,CHEMBL4649948,PERCENTEFFECT,%,UNCHECKED,ORGANISM,86589,1.0,quantitative,25.0,4405.0,0.051,86589.0,NaN,NaN,NaN,True,0.729,0.010,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN
1,CHEMBL4649948,PERCENTEFFECT,%,UNCHECKED,ORGANISM,86589,1.0,quantitative,50.0,1268.0,0.015,86589.0,NaN,NaN,NaN,True,0.700,0.017,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN
2,CHEMBL4649948,PERCENTEFFECT,%,UNCHECKED,ORGANISM,86589,1.0,quantitative,75.0,361.0,0.004,86589.0,NaN,NaN,NaN,True,0.646,0.020,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN
3,CHEMBL4649949,PERCENTEFFECT,%,UNCHECKED,ORGANISM,86575,1.0,quantitative,25.0,10324.0,0.119,86575.0,NaN,NaN,NaN,True,0.699,0.007,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN
4,CHEMBL4649949,PERCENTEFFECT,%,UNCHECKED,ORGANISM,86575,1.0,quantitative,50.0,2181.0,0.025,86575.0,NaN,NaN,NaN,True,0.740,0.011,False,NaN,NaN,False,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,CHEMBL4011046,GI,%,ORGANISM,ORGANISM,103,1.0,quantitative,50.0,103.0,1.000,103.0,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,True,1.0,0.0,False,NaN,NaN
97,CHEMBL4011046,GI,%,ORGANISM,ORGANISM,103,1.0,quantitative,75.0,103.0,1.000,103.0,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,True,1.0,0.0,False,NaN,NaN
98,CHEMBL4011042,GI,%,ORGANISM,ORGANISM,103,1.0,quantitative,25.0,103.0,1.000,103.0,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,True,1.0,0.0,False,NaN,NaN
99,CHEMBL4011042,GI,%,ORGANISM,ORGANISM,103,1.0,quantitative,50.0,103.0,1.000,103.0,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,True,1.0,0.0,False,NaN,NaN


In [ ]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import sys
import os
from collections import defaultdict

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.width", 2000)
pd.set_option("display.max_colwidth", None)

display(HTML("""
<style>
.dataframe td, .dataframe th {
    white-space: nowrap !important;
}
</style>
"""))

In [ ]:
# Define root directory
# root = os.path.dirname(os.path.abspath(__file__))
root = "."
sys.path.append(os.path.join(root, "..", "src"))
from default import DATAPATH, CONFIGPATH

# Load pathogen info
# pathogen_code = sys.argv[1]
pathogen_code = 'mtuberculosis'
df = pd.read_csv(os.path.join(CONFIGPATH, 'pathogens.csv'))
row = df.loc[df["code"].eq(pathogen_code)]
if row.empty: 
    raise SystemExit(f"Unknown code: {pathogen_code}")
pathogen = row.iloc[0]["pathogen"]

# Create output directory
OUTPUT = os.path.join(root, "..", "output")

# Shared columns
KEYS = ["assay_id", "activity_type", "unit"]

# Columns to take from each table
COLUMNS_CLEANED = ["assay_id", "assay_type", "assay_organism", "doc_chembl_id", "target_type", "target_chembl_id", "target_organism", "activity_type", 
                "unit", "canonical_unit", "activities", "nan_values", "cpds", "direction", "activity_comment_counts", "standard_text_count"]
COLUMNS_CLUSTERS = ['clusters_0.3', 'clusters_0.6', 'clusters_0.85']
COLUMNS_DATASETS = ["equal", 'higher', 'lower', "dataset_type", "cpds_qt", "min_", "p1", "p25", "p50", "p75", "p99", "max_", "pos_ql", "ratio_ql", "cpds_ql"]

In [ ]:
# Load assays info
ASSAYS_CLEANED = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_cleaned.csv"))
ASSAYS_CLUSTERS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_clusters.csv"))
ASSAYS_PARAMETERS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_parameters.csv"))
ASSAYS_DATASETS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_datasets.csv"))

# Get assay to quantitative data info
assay_to_qt_info = defaultdict(list)
for assay_id, activity_type, unit, expert_cutoff, ratio_qt in ASSAYS_DATASETS[['assay_id', 'activity_type', 'unit', 'expert_cutoff', 'ratio_qt']].values:
    assay_to_qt_info[tuple([assay_id, activity_type, unit])].append([expert_cutoff, ratio_qt])

# Unique row per assay
ASSAYS_DATASETS = ASSAYS_DATASETS[KEYS + COLUMNS_DATASETS].drop_duplicates().reset_index(drop=True)

# Get cutoffs and ratios
cutoffs = [";".join([str(j[0]) for j in assay_to_qt_info[tuple(i)]]) for i in ASSAYS_DATASETS[['assay_id', 'activity_type', 'unit']].values]
ratios = [";".join([str(j[1]) for j in assay_to_qt_info[tuple(i)]]) for i in ASSAYS_DATASETS[['assay_id', 'activity_type', 'unit']].values]

# Store results
ASSAYS_DATASETS.insert(8, 'cutoffs', cutoffs)
ASSAYS_DATASETS.insert(9, 'ratios', ratios)

In [ ]:
len(ASSAYS_CLEANED), len(ASSAYS_CLUSTERS), len(ASSAYS_PARAMETERS), len(ASSAYS_DATASETS)

In [ ]:
ASSAYS_DATASETS

In [ ]:
ASSAYS_DATASETS